In [1]:
import pandas as pd

In [2]:
import pathlib as pl

In [3]:
data_folder = pl.Path('data')

In [23]:
df = pd.read_parquet(data_folder / 'yellow_tripdata_2024-01.parquet', columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance'])

In [10]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance
0,2024-01-01 00:57:55,2024-01-01 01:17:43,1.72
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.80
2,2024-01-01 00:17:06,2024-01-01 00:35:01,4.70
3,2024-01-01 00:36:38,2024-01-01 00:44:56,1.40
4,2024-01-01 00:46:51,2024-01-01 00:52:57,0.80


In [11]:
df['trip-length-time'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [13]:
df.dtypes

tpep_pickup_datetime      datetime64[us]
tpep_dropoff_datetime     datetime64[us]
trip_distance                    float64
trip-length-time         timedelta64[us]
dtype: object

In [12]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,trip-length-time
0,2024-01-01 00:57:55,2024-01-01 01:17:43,1.72,0 days 00:19:48
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.80,0 days 00:06:36
2,2024-01-01 00:17:06,2024-01-01 00:35:01,4.70,0 days 00:17:55
3,2024-01-01 00:36:38,2024-01-01 00:44:56,1.40,0 days 00:08:18
4,2024-01-01 00:46:51,2024-01-01 00:52:57,0.80,0 days 00:06:06


In [38]:
import pandas as pd
import pathlib as pl

# Load the existing data
data_folder = pl.Path('data')
df = pd.read_parquet(data_folder / 'yellow_tripdata_2024-01.parquet', columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance'])

# Load the new data
new_data = pd.read_parquet(data_folder / 'yellow_tripdata_2023-12.parquet', columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance'])

In [33]:
df.isnull().sum(axis = 0)

tpep_pickup_datetime     0
tpep_dropoff_datetime    0
trip_distance            0
dtype: int64

In [39]:
df['tpep_pickup_datetime'].max()

Timestamp('2024-02-01 00:01:15')

In [40]:
df['tpep_pickup_datetime'].min()

Timestamp('2002-12-31 22:59:39')

In [26]:
df.sort_values('tpep_dropoff_datetime').head(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance
53119,2002-12-31 22:59:39,2002-12-31 23:05:41,0.63
53120,2002-12-31 22:59:39,2002-12-31 23:05:41,0.63
2558056,2009-01-01 00:24:09,2009-01-01 01:13:00,10.88
1230169,2009-01-01 23:30:39,2009-01-02 00:01:39,10.99
10915,2009-01-01 23:58:40,2009-01-02 00:01:40,0.46
369,2023-12-31 23:39:17,2023-12-31 23:42:00,0.47
753,2023-12-31 23:41:02,2023-12-31 23:48:03,0.40
3176,2023-12-31 23:47:28,2023-12-31 23:57:07,1.44
2615,2023-12-31 23:56:45,2024-01-01 00:00:28,0.97
2210,2023-12-31 23:57:17,2024-01-01 00:01:50,0.53


In [36]:
# Append the new data to the existing DataFrame
df = pd.concat([new_data, df])
df['trip_length_time'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
# Calculate the trip length for each trip
df['trip_length_time'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60  # in minutes
df['trip_length_distance'] = df['trip_distance']  # in miles

# Set the pickup datetime as the index for rolling calculation
df.set_index('tpep_pickup_datetime', inplace=True)
df = df.sort_index()

# Calculate the rolling 45-day average trip length
rolling_avg_time = df['trip_length_time'].rolling('45D').mean()
rolling_avg_distance = df['trip_length_distance'].rolling('45D').mean()

print(f"The 45-day rolling average trip length in time is:\n{rolling_avg_time}")
print(f"The 45-day rolling average trip length in distance is:\n{rolling_avg_distance}")

The 45-day rolling average trip length in time is:
tpep_pickup_datetime
2002-12-31 22:16:54    1005.233333
2002-12-31 22:59:39     505.633333
2002-12-31 22:59:39     339.100000
2002-12-31 23:03:18     585.933333
2009-01-01 00:02:41    1060.783333
                          ...     
2024-01-31 23:59:54      16.175183
2024-01-31 23:59:55      16.175182
2024-02-01 00:00:17      16.174709
2024-02-01 00:00:39      16.174705
2024-02-01 00:01:15      16.174717
Name: trip_length_time, Length: 6341191, dtype: float64
The 45-day rolling average trip length in distance is:
tpep_pickup_datetime
2002-12-31 22:16:54    3.250000
2002-12-31 22:59:39    1.940000
2002-12-31 22:59:39    1.503333
2002-12-31 23:03:18    8.175000
2009-01-01 00:02:41    1.370000
                         ...   
2024-01-31 23:59:54    3.670489
2024-01-31 23:59:55    3.670488
2024-02-01 00:00:17    3.670471
2024-02-01 00:00:39    3.670462
2024-02-01 00:01:15    3.670460
Name: trip_length_distance, Length: 6341191, dtype: float64